# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset, Model
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
import numpy as np
import joblib, pickle

## Dataset

### Overview
The dataset I'm using tracks car mpg as a function of several car characteristics like weight and acceleration. The task performed is regression to predict mpg as a function of these characteristics.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
subscription_id = 'b19c9c81-5f59-4537-b3c0-c1beb163ec22'
resource_group = 'aml-quickstarts-129321'
workspace_name = 'quick-starts-ws-129321'

ws = Workspace.get(name="quick-starts-ws-129321")
exp = Experiment(workspace=ws, name="Ranga")

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='mpg')
ds = dataset.to_pandas_dataframe()

ds = ds.replace('?',0)
ds.dropna(inplace=True)

ds.drop(columns=['car name'],inplace=True)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ACJJBCXP9 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


## AutoML Configuration

I chose the regression task since that represents the nature of this problem, and R-squared is correspondingly a good metric. R-squared is also the default scoring metric for GradientBoosting Regressor, which is the model that I'm using in Hyperparameter tuning.

In [3]:
# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=20,
    task='regression',
    primary_metric='r2_score',
    training_data=ds,
    label_column_name='mpg',
    n_cross_validations=3)

In [4]:
# TODO: Submit your experiment
autoexp = Experiment(workspace=ws,name='AutoRanga')
remote_run = autoexp.submit(automl_config)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       DONE
DESCRIPTION:  High cardinality features were detected in your inputs and handled.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization
DETAILS:      High cardinality features refer to columns that contain a large percentage of unique values.
+---------------------------------+---------------------------------+
|Column name                      |Column Content Type              |
+=============================

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          0:00:30       0.7822    0.7822
         1   MaxAbsScaler XGBoostRegressor                  0:00:24       0.8192    0.8192
         2   StandardScalerWrapper XGBoostRegressor         0:00:25       0.8308    0.8308
         3   StandardScalerWrapper RandomForest             0:00:25      

{'runId': 'AutoML_c1ff0474-a355-41c9-8bb5-3317faf8de83',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-12-05T14:23:35.44746Z',
 'endTimeUtc': '2020-12-05T14:45:04.492886Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'r2_score',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoRanga","subscription_id":"b19c9c81-5f59-4537-b3c0-c1beb163ec22","resource_group":"aml-quickstarts-129321","workspace_name":"quick-starts-ws-129321","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"r2_score","task_type":"regression","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,

## Run Details

The models trained feature a variety of preprocessing methods i.e. scaling to 0-1, scaling by max values, and normalization. After this preparation, a variety of models are fit. What we find is that simple stochastic gradient descents and decision trees are largely inadequate. Random Forest uses many more trees (the 'bagging' technique) with many leaves of equal weight. Not surprisingly, adding 'boosting' to bagging through XGBoost results in improved performance since we penalize leaves that don't improve model performance.
TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [5]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [6]:
best,fitted = remote_run.get_output()
print(best)

Run(Experiment: AutoRanga,
Id: AutoML_c1ff0474-a355-41c9-8bb5-3317faf8de83_45,
Type: None,
Status: Completed)


In [7]:
print(fitted)

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         

In [9]:
fitted.steps[-1]

('prefittedsoftvotingregressor',
 PreFittedSoftVotingRegressor(estimators=[('42',
                                           Pipeline(memory=None,
                                                    steps=[('sparsenormalizer',
                                                            <azureml.automl.runtime.shared.model_wrappers.SparseNormalizer object at 0x7f479bd2d9b0>),
                                                           ('xgboostregressor',
                                                            XGBoostRegressor(base_score=0.5,
                                                                             booster='gbtree',
                                                                             colsample_bylevel=1,
                                                                             colsample_bynode=1,
                                                                             colsample_bytree=1,
                                                              

In [10]:
#TODO: Save the best model
filename = 'best_automl_model.sav'
# pickle.dump(fitted, open(filename, 'wb'))

joblib.dump(fitted,filename)

['best_automl_model.sav']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [11]:
registeredfinal = Model.register(workspace=ws,model_path='./best_automl_model.sav',model_name='registered.sav')

Registering model registered.sav


In [13]:
#Create inference config
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(workspace=ws,name='AzureML-AutoML')

inferencefinal = InferenceConfig(entry_script='score.py',environment=env) 

#Deploy model and check service state
from azureml.core.webservice import LocalWebservice, AciWebservice


deployfinal = AciWebservice.deploy_configuration()
svcs = Model.deploy(workspace=ws,name='deploy',models=[registeredfinal],inference_config=inferencefinal,deployment_config=deployfinal)
svcs.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.....................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [14]:
print(svcs.state)

Healthy


In [15]:
print(svcs.scoring_uri)

http://29e036ea-6c5a-48e6-ad64-5464eb02ec6c.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [16]:
import requests
import json

# URL for the web service
scoring_uri = 'http://29e036ea-6c5a-48e6-ad64-5464eb02ec6c.southcentralus.azurecontainer.io/score'


data = {'data':[{
    'cylinders':4,
    'displacement':84,
    'horsepower':65,
    'weight':2110,
    'acceleration':12,
    'model yr':80,
    'origin':3}
]
        }

input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}


# # Make the request and display the response
result = requests.post(scoring_uri,input_data,headers=headers)
print("prediction:",result.text)

prediction: [37.98189931616876]


In [18]:
env.save_to_directory('./training',overwrite=False)

TODO: In the cell below, print the logs of the web service and delete the service

In [19]:
print(svcs.get_logs())

2020-12-05T14:57:54.8682742Z stdout F 2020-12-05T14:57:54,706579800+00:00 - iot-server/run 
2020-12-05T14:57:54.9463202Z stdout F 2020-12-05T14:57:54,784756300+00:00 - gunicorn/run 
2020-12-05T14:57:54.9752787Z stdout F 2020-12-05T14:57:54,812926400+00:00 - nginx/run 
2020-12-05T14:57:55.0473033Z stdout F 2020-12-05T14:57:54,813868900+00:00 - rsyslog/run 
2020-12-05T14:57:55.0743102Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_7cf2b5eebfc9a832d561ce064cfbc38b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-12-05T14:57:55.1577944Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_7cf2b5eebfc9a832d561ce064cfbc38b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-12-05T14:57:55.1618084Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_7cf2b5eebfc9a832d561ce064cfbc38b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-12-05T14:57:55.1618084Z stderr F /usr/sbin/nginx:

In [ ]:
svcs.delete()